# Build the PQCs for CDF and PDF

This notebook provides instructions for building the Parametric Quantum Circuits (**PQC**) for computing **CDF** and their corresponding **PDF**


In [ ]:
import sys 
sys.path.append("../../")
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1. PQC for computing CDFs


The fundamental idea is to use a **PQC** as a surrogate model, denoted $F^*(\textbf{x}, \theta)$, to approximate a more complex and computationally expensive financial distribution $F(\textbf{x})$. This surrogate model allows for efficient Value at Risk (VaR) computations.

In this approach, the **PQC** is characterized by a unitary operator $U(\textbf{x}, \theta)$, where $\textbf{x}$ represents the input features, and $\theta$ are the trainable parameters. The evaluation of the CDF for a given input $\textbf{x}$ is performed by measuring an observable $M$ after applying the parametric quantum circuit. Mathematically, the CDF approximation is given by:

$$F^*(\textbf{x})=\langle{0}|U^{\dagger}(\textbf{x};\theta )MU(\textbf{x}; \theta) |0\rangle$$


Under the module **QQuantLib.qml4var.architectures** several examples of **PQC** architectures and **Observables** were implemented into the **EVIDEN myqlm** software. In this notebook we review the two following functions from this module.

* hardware_efficient_ansatz
* z_observable






### 1.1 hardware_efficient_ansatz


The *hardware_efficient_ansatz* function from **QQuantLib.qml4var.architectures** module constructs a hardware-efficient ansatz **PQC**. The architecture consists of multiple layers, where each layer alternates between parametrized single-qubit rotations and controlled-NOT (CNOT) gates arranged in a ladder configuration. The rotation gates serve different purposes:

* $R_X$ gates: These are used to encode the trainable weights ($\theta$). 
* $R_Y$ gates: These are used to encode the input features ($\textbf{x}$).

The *hardware_efficient_ansatz* function receives a keyword arguments that allow configuring the **PQC**. Mandatory keys are:

* features_number : number of input features.
* n_qubits_by_feature : number of qubits used for encoding each possible feature.
* n_layers : number of layers the ansatz will have.
* base_frecuency : slope for feature normalization (applied to $R_Y$)
* shift_feature : shift for feature normalization (applied to $R_Y$)

The function returns:

* QLM Program implementation of the desired **PQC**
* weights_names : list with names used in the **PQC** for the weights
* features_names : list with names used in the **PQC** for the features.

In [ ]:
from QQuantLib.qml4var.architectures import hardware_efficient_ansatz

In [ ]:
pqc_cfg = {
    "features_number" : 1,
    "n_qubits_by_feature" : 2,
    "n_layers": 2    
}
pqc_cfg.update({
    "base_frecuency" : [1.0] * pqc_cfg["features_number"],
    "shift_feature" : [0.0] * pqc_cfg["features_number"],
})


In [ ]:
pqc, weights_names, features_names = hardware_efficient_ansatz(**pqc_cfg)

In [ ]:
# plot PQC
circuit = pqc.to_circ()
circuit.display()

As can be seen in the circuit of the Figure, the parametric gates of the **PQC** have well-defined names depending if the parameter is a trainable weight (*weights* strings like) or an input feature (*feature* strings like).

**BE AWARE**
The user can build its own **PQC** architectures but always has to differentiate what of the **PQC** parameters are trainable weights and what are input features. Additionally, two different Python lists should be generated: a list with the name of the weights (*weights_names* variable) and another with the name of the input features (*features_names*)

In [ ]:
#Names of the PQC parameters related to trainable weights
print(weights_names)
#Names of the PQC parameters related to input features
print(features_names)

#### base_frecuency and shift_feature keywords

To have a better trainable **PQC** it is useful to normalize the features. This can be done by providing to the *hardware_efficient_ansatz* the *base_frecuency* and the *shift_feature* arguments. The value provided to the gate that implements the feature encoding will be then:

$$base\_frecuency * feature + shift\_feature$$

To compute the *base_frecuency* and the *shift_feature* the *normalize_data* function from the **QQuantLib.qml4var.architectures** module can be used. The inputs of this function are:

* min_value: list with minimum values that the features can take.
* max_value: list with maximum values that the features can take.
* min_x: list with the values of the feature encoding gate corresponding to the minimum values of the features.
* max_x: list with the values of the feature encoding gate corresponding to the minimum values of the features.

In [ ]:
from QQuantLib.qml4var.architectures import normalize_data

In [ ]:
# In this case:
# min-> -0.5 pi
minval =0.1
# max-> 0.5 pi
maxval = 3.0
base_frecuency, shift_feature = normalize_data(
    [minval] * pqc_cfg["features_number"],
    [maxval] * pqc_cfg["features_number"],
    [-0.5*np.pi] * pqc_cfg["features_number"],
    [0.5*np.pi] * pqc_cfg["features_number"]   
)

In [ ]:
# Update with the normalization info
pqc_cfg.update({
    "base_frecuency" : base_frecuency,
    "shift_feature" : shift_feature    
})   
print(pqc_cfg)

In [ ]:
pqc, weights_names, features_names = hardware_efficient_ansatz(**pqc_cfg)

In [ ]:
# plot PQC
circuit = pqc.to_circ()
circuit.display()

### 1.2 The Observable

Once we have defined the **PQC** we need to build the desired observable to compute. This can be with the *z_observable* from *architectures*. In this case, we create a $Z$ measurment for all the qubits of the **PQC**. The function need as arguments:

* features_number
* n_qubits_by_feature

that will be used for computing the total number of qubits of the **PQC**. The same keywords that can be provided to *hardware_efficient_ansatz* can be provided to the *z_observable*

The user can define their own observables but should be passed the total number of qubits of the **PQC**

In [ ]:
from QQuantLib.qml4var.architectures import z_observable

In [ ]:
observable = z_observable(**pqc_cfg)

In [ ]:
observable

## 2. Computing the PDF.

In addition to a **PQC** for computing the **CDF** ($F^*(\textbf{x}, \theta)$), our training workflow will need to compute the corresponding **PDF** of the **CDF**: 

$$f^*(\textbf{x}, \theta) = \frac{\partial^m F^*(\textbf{x}, \theta)}{\partial x_{m-1} \cdots \partial x_1 \partial x_0}$$

To compute these derivatives for an input **PQC** the parameter shift rule should be applied to the **features** parameters consecutively (so first we apply the parameter shift rule to the the first *feature* parameter obtaining several circuits. Over each obtained circuit we need to apply the parameter shift rule to the second *feature* parameter and so on). 

The *compute_pdf_from_pqc* function from **QQuantLib.qml4var.architectures** builds all these mandatory circuits for a given **PQC** and the desired parameter features.

This function needs:

* batch: QLM Batch. A batch with the job with the **PQC** and the **Observable**.
* parameters: list with the names of the *features* parameters of the **PQC** for obtaining the derivatives.

The functions returns a QLM Batch with all the jobs with quantum circuits mandatory for computing the **PDF**. 

**BE AWARE**

The jobs of the returned Batch computes directly the expected value with the corresponding multiplicative factors. So for a fixed input and weights, the user only needs to sum up all the values returned for each executed job for computing the corresponding **PDF** evaluation.

In [ ]:
from QQuantLib.qml4var.architectures import compute_pdf_from_pqc
from qat.core import Batch

In [ ]:
# Generate the Batch
job = Batch([pqc.to_circ().to_job(observable=observable)])

In [ ]:
# Build the jobs for computing the PDF
pdf_jobs = compute_pdf_from_pqc(job, features_names)

In [ ]:
# Obtained Circuit
for job in pdf_jobs:
    job.circuit().display()